In [1]:
import pandas as pd
import numpy as np
from os.path import join
import os
import re
from goatools import obo_parser
from pubchempy import *

import warnings
warnings.filterwarnings('ignore')

## 1. Reading information about all GO Terms:

We downloaded the "go.obo" file with information about all GO Terms from: http://geneontology.org/docs/download-ontology/

### (a) Storing the definition, name, and ID of all GO terms in a pandas DataFrame:

In [2]:
filename_go_obo = join("..", "..", "data", "GOA", "go_terms", 'go.obo')
obo_reader = obo_parser.OBOReader(filename_go_obo)
df = pd.DataFrame(columns = ["GO ID", "Definition", "Name"])

file1 = open(filename_go_obo, 'r')
Lines = file1.read()
start = 0
while start != -1:
    start = Lines.index('[Term]\n')
    Lines = Lines[start+1:]
    GO_Term = Lines[: Lines.index('[Term]\n')]
    definition =  GO_Term[GO_Term.index("\ndef")+6:]
    definition = definition[:definition.index("\n")]
    name = GO_Term[GO_Term.index("\nname")+7:]
    name = name[:name.index("\n")]
    namespace = GO_Term[GO_Term.index("\nnamespace")+12:]
    namespace = namespace[:namespace.index("\n")]
    
    ID = GO_Term[GO_Term.index("\nid")+5:]
    ID = ID[:ID.index("\n")]
    
    df = df.append({"GO ID" : ID , "Definition" : definition, "Name" : name, "Namespace": namespace}, ignore_index = True)

ValueError: substring not found

In [3]:
df.to_pickle(join("..", "..", "data", "GOA", "go_terms", "all_GO_terms.pkl"))
df.head()

,GO ID,Definition,Name,Namespace
0,GO:0000001,"""The distribution of mitochondria, including t...",mitochondrion inheritance,biological_process
1,GO:0000002,"""The maintenance of the structure and integrit...",mitochondrial genome maintenance,biological_process
2,GO:0000003,"""The production of new individuals that contai...",reproduction,biological_process
3,GO:0000005,"""OBSOLETE. Assists in the correct assembly of ...",obsolete ribosomal chaperone activity,molecular_function
4,GO:0000006,"""Enables the transfer of zinc ions (Zn2+) from...",high-affinity zinc transmembrane transporter a...,molecular_function


### (b) Removing all GO terms without information abuot transporter:

In [4]:
droplist = []
for ind in df.index:
    name, definition = df["Name"][ind].lower(), df["Definition"][ind].lower()
    if (not "transport" in name and not "abc-type" in name and not "permease" in name 
         and not "transport" in definition and not "abc-type" in definition and not "permease" in definition):
        droplist.append(ind)
df.drop(droplist, inplace = True)
df.reset_index(inplace = True, drop = True)
df.head()

,GO ID,Definition,Name,Namespace
0,GO:0000006,"""Enables the transfer of zinc ions (Zn2+) from...",high-affinity zinc transmembrane transporter a...,molecular_function
1,GO:0000007,"""Enables the transfer of a solute or solutes f...",low-affinity zinc ion transmembrane transporte...,molecular_function
2,GO:0000017,"""The directed movement of alpha-glucosides int...",alpha-glucoside transport,biological_process
3,GO:0000039,"""OBSOLETE. (Was not defined before being made ...",obsolete plasma membrane long-chain fatty acid...,molecular_function
4,GO:0000041,"""The directed movement of transition metal ion...",transition metal ion transport,biological_process


### (c) Removing all regulation proteins:

In [5]:
droplist = []
for ind in df.index:
    if  "regulation" in df["Name"][ind].lower():
        droplist.append(ind)
        
df.drop(droplist, inplace = True)
len(droplist)
df

,GO ID,Definition,Name,Namespace
0,GO:0000006,"""Enables the transfer of zinc ions (Zn2+) from...",high-affinity zinc transmembrane transporter a...,molecular_function
1,GO:0000007,"""Enables the transfer of a solute or solutes f...",low-affinity zinc ion transmembrane transporte...,molecular_function
2,GO:0000017,"""The directed movement of alpha-glucosides int...",alpha-glucoside transport,biological_process
3,GO:0000039,"""OBSOLETE. (Was not defined before being made ...",obsolete plasma membrane long-chain fatty acid...,molecular_function
4,GO:0000041,"""The directed movement of transition metal ion...",transition metal ion transport,biological_process
...,...,...,...,...
3005,GO:2001103,"""The directed movement of a maltohexaoseacetat...",maltohexaose transport,biological_process
3006,GO:2001104,"""The directed movement of a heptasaccharideace...",heptasaccharide transport,biological_process
3007,GO:2001105,"""The directed movement of a maltoheptaoseaceta...",maltoheptaose transport,biological_process
3011,GO:2001142,"""The directed movement of a nicotinateacetate ...",nicotinate transport,biological_process


## 2. Mapping substrate names to metabolite IDs:

### (a) Extracting substrate names:

In [6]:
starter = ["obsolete ", 
           "high-affinity secondary active ", "low-affinity ", "secondary active ",
           "high-affinity ", "low-affinity ", "abc-type ", "proton-dependent ", "atpase-coupled ",
          "mitochondrion to", "mitochondrial "]
endings = [" secondary active transmembrane transporter activity",
           " transmembrane transporter activity",
           " transporter activity"
           " transmembrane transporter",
           " transmembrane transport",
           " activity",
          " transporter",
          " transport",
          " transfer",
          " autotransporter",
          " channel activity"]


def get_substrate(name):
    for end in endings:
        name = name.replace(end, "")
    for start in starter:
        name = name.replace(start, "")
    if "import into cell" in name:
        name = name[:name.find(" involved")]
    return(name)

def ends_with(full_string, sub_string):
    if not sub_string in full_string:
        return(False)
    if full_string[full_string.find(sub_string): ] == sub_string:
        return(True)
    else:
        return(False)

In [7]:
df["substrate"] = ""


transmembrane_transporter_activity = []
for ind in df.index:
    name = df["Name"][ind].lower()
    if ends_with(full_string = name, sub_string = "transporter activity"):
        transmembrane_transporter_activity.append(name)
        substrate = get_substrate(name = name)
    elif ends_with(full_string = name, sub_string = " transfer activity"):
        transmembrane_transporter_activity.append(name)
        substrate = get_substrate(name = name)
    elif ends_with(full_string = name, sub_string = "transport"):
        transmembrane_transporter_activity.append(name)
        substrate = get_substrate(name = name)
    elif ends_with(full_string = name, sub_string = "transporter"):
        transmembrane_transporter_activity.append(name)
        substrate = get_substrate(name = name)
    elif ends_with(full_string = name, sub_string = "channel activity"):
        transmembrane_transporter_activity.append(name)
        substrate = get_substrate(name = name)
    else: 
        substrate = ""
    try:
        if substrate[0] == " ":
            substrate = substrate[1:]
    except IndexError: pass
    df["substrate"][ind] = substrate

df = df.loc[df["substrate"] != ""]

df.to_pickle(join("..", "..", "data", "GOA", "go_terms", "df_GO_with_substrates.pkl"))

### (b)  Mapping substrates to IDs:

In [8]:
df_unmapped = pd.DataFrame({"metabolites" : list(set(list(df["substrate"])))})

#### (b)(i) Mapping to  KEGG Compound IDs

In [9]:
drugs_df = pd.read_pickle(join("..", "..", "data", "substrates", "KEGG_drugs_df.pkl"))
compounds_df = pd.read_pickle(join("..", "..", "data", "substrates",  "KEGG_substrate_df.pkl"))
KEGG_substrate_df = compounds_df.append(drugs_df).reset_index(drop = True)

##If we have multiple IDs for the same substrate name, we keep the first ID:
droplist = []
for ind in KEGG_substrate_df.index:
    if not ind in droplist:
        substrate = KEGG_substrate_df["substrate"][ind]
        help_df = KEGG_substrate_df.loc[KEGG_substrate_df["substrate"] == substrate]
        if len(help_df) > 1 :
            droplist = droplist + list(help_df.index)[1:]

KEGG_substrate_df.drop(droplist, inplace = True)

KEGG_substrate_df["substrate"] = [name.lower() for name in KEGG_substrate_df["substrate"]]
df_unmapped["substrate"] = [name.lower() for name in df_unmapped["metabolites"]]

df_unmapped = df_unmapped.merge(KEGG_substrate_df, on = "substrate", how = "left")
print("For %s out of %s substrates, we could not map the substrate name to a KEGG ID." %
      (sum(pd.isnull(df_unmapped["KEGG ID"])), len(df_unmapped)))

For 604 out of 927 substrates, we could not map the substrate name to a KEGG ID.


#### (b)(ii) Mapping to PubChem IDs

In [10]:
def get_ID_from_name(name):
    cs = get_compounds(name, 'name')
    inchi, cid = np.nan, np.nan
    
    for c in cs:
        
        try: inchi = c.inchi
        except AttributeError: pass
        
        try: cid = c.cid
        except AttributeError: pass
        
        if not pd.isnull(inchi) and not pd.isnull(cid):
            return(inchi, cid)
    return(inchi, cid)

In [11]:
df_unmapped["PubChem CID"] = np.nan
df_unmapped["InChI"] = np.nan

In [12]:
for ind in df_unmapped.index:
    if ind > -1:
        if pd.isnull(df_unmapped["KEGG ID"][ind]):
            df_unmapped["InChI"][ind], df_unmapped["PubChem CID"][ind] = get_ID_from_name(name = df_unmapped["substrate"][ind])

In [13]:
df_unmapped = df_unmapped.drop(columns = ["metabolites"])
df_unmapped

,substrate,KEGG ID,PubChem CID,InChI
0,fluoride,C00742,NaN,NaN
1,transepithelial ammonium,NaN,NaN,NaN
2,4-(trimethylammonio)butanoate,NaN,725.0,"InChI=1S/C7H15NO2/c1-8(2,3)6-4-5-7(9)10/h4-6H2..."
3,methyl-driven active,NaN,NaN,NaN
4,peptide-acetyl-coa,NaN,NaN,NaN
...,...,...,...,...
922,recycling endosome to golgi,NaN,NaN,NaN
923,gap junction-mediated intercellular,NaN,NaN,NaN
924,malonate(1-),NaN,3748644.0,"InChI=1S/C3H4O4/c4-2(5)1-3(6)7/h1H2,(H,4,5)(H,..."
925,aluminum ion,NaN,104727.0,InChI=1S/Al/q+3


#### (b)(iii) Mapping all substrate IDs to CHEBI IDs

First, we create a txt files with all KEGG CIDs and alls PubChem CIDs:

In [14]:
#create txt file with all CIDs in matches:
all_KEGG_IDs = list(set(df_unmapped["KEGG ID"].loc[~pd.isnull(df_unmapped["KEGG ID"])]))

f = open(join("..", "..", "data", "GOA", "go_terms", "all_KEGG_CIDs.txt"),"w") 
for cid in all_KEGG_IDs:
    f.write(str(cid) + "\n")
f.close()

#create txt file with all CIDs in matches:
all_PubChem_IDs = list(set(df_unmapped["PubChem CID"].loc[~pd.isnull(df_unmapped["PubChem CID"])]))

f = open(join("..", "..", "data", "GOA", "go_terms", "all_PubChem_CIDs.txt"),"w") 
for cid in all_PubChem_IDs:
    f.write(str(int(cid)) + "\n")
f.close()

The txt-files can be used as the input for the webservice http://csbg.cnb.csic.es/mbrole2/conversion.php to map the the IDs to CHEBI IDs. 

In [15]:
KEGG_to_CHEBI = pd.read_csv(join("..", "..", "data", "GOA", "go_terms",  "mbrole2_conversion_KEGG.tsv"), sep= "\t")
KEGG_to_CHEBI.rename(columns = {"Input" : "KEGG ID", "Output" : "ChEBI"}, inplace = True)
KEGG_to_CHEBI.drop(columns = ["Input_source", "Output_source"], inplace = True)
KEGG_to_CHEBI

df_unmapped["ChEBI"] = np.nan
for ind in df_unmapped.index:
    CID = df_unmapped["KEGG ID"][ind]
    try:
        df_unmapped["ChEBI"][ind] = list(KEGG_to_CHEBI["ChEBI"].loc[KEGG_to_CHEBI["KEGG ID"] == CID])[0]
    except IndexError:
        pass

In [16]:
Pubchem_to_CHEBI = pd.read_csv(join("..", "..", "data", "GOA", "go_terms",  "mbrole2_conversion_Pubchem.tsv"), sep= "\t")
Pubchem_to_CHEBI.rename(columns = {"Input" : "PubChem CID", "Output" : "ChEBI"}, inplace = True)
Pubchem_to_CHEBI.drop(columns = ["Input_source", "Output_source"], inplace = True)
Pubchem_to_CHEBI

for ind in df_unmapped.index:
    if pd.isnull(df_unmapped["ChEBI"][ind]):
        try:
            CID = int(df_unmapped["PubChem CID"][ind])
            df_unmapped["ChEBI"][ind] = list(Pubchem_to_CHEBI["ChEBI"].loc[Pubchem_to_CHEBI["PubChem CID"] == CID])[0]
        except:
            pass

In [17]:
df_unmapped.loc[~pd.isnull(df_unmapped["ChEBI"])]

,substrate,KEGG ID,PubChem CID,InChI,ChEBI
0,fluoride,C00742,NaN,NaN,CHEBI:5113
2,4-(trimethylammonio)butanoate,NaN,725.0,"InChI=1S/C7H15NO2/c1-8(2,3)6-4-5-7(9)10/h4-6H2...",CHEBI:16244
5,tartrate,C00898,NaN,NaN,CHEBI:30924
6,aerobactin,C05554,NaN,NaN,CHEBI:2499
10,pyridoxal phosphate,C00018,NaN,NaN,CHEBI:26424
...,...,...,...,...,...
916,oleate,C00712,NaN,NaN,CHEBI:25664
917,ceramide,C00195,NaN,NaN,CHEBI:52573
924,malonate(1-),NaN,3748644.0,"InChI=1S/C3H4O4/c4-2(5)1-3(6)7/h1H2,(H,4,5)(H,...",CHEBI:30795
925,aluminum ion,NaN,104727.0,InChI=1S/Al/q+3,CHEBI:49470


In [18]:
df = df.merge(df_unmapped, how = "left", on = "substrate")

df_GO_metabolite =df.loc[~pd.isnull(df["ChEBI"])]
df_GO_metabolite

,GO ID,Definition,Name,Namespace,substrate,KEGG ID,PubChem CID,InChI,ChEBI
0,GO:0000006,"""Enables the transfer of zinc ions (Zn2+) from...",high-affinity zinc transmembrane transporter a...,molecular_function,zinc,NaN,23994.0,InChI=1S/Zn,CHEBI:27363
1,GO:0000007,"""Enables the transfer of a solute or solutes f...",low-affinity zinc ion transmembrane transporte...,molecular_function,zinc ion,C00038,NaN,NaN,CHEBI:10113
5,GO:0000064,"""Enables the transfer of L-ornithine from one ...",L-ornithine transmembrane transporter activity,molecular_function,l-ornithine,C00077,NaN,NaN,CHEBI:6280
6,GO:0000095,"""Enables the transfer of S-adenosylmethionine ...",S-adenosyl-L-methionine transmembrane transpor...,molecular_function,s-adenosyl-l-methionine,C00019,NaN,NaN,CHEBI:22036
10,GO:0000102,"""Enables the transfer of L-methionine from one...",L-methionine secondary active transmembrane tr...,molecular_function,l-methionine,C00073,NaN,NaN,CHEBI:6271
...,...,...,...,...,...,...,...,...,...
1501,GO:2001097,"""The directed movement of a laminaritrioseacet...",laminaritriose transport,biological_process,laminaritriose,NaN,11477625.0,InChI=1S/C18H32O16/c19-1-4-7(22)10(25)11(26)17...,CHEBI:55514
1503,GO:2001099,"""The directed movement of a maltotetraoseaceta...",maltotetraose transport,biological_process,maltotetraose,C02052,NaN,NaN,CHEBI:6671
1507,GO:2001103,"""The directed movement of a maltohexaoseacetat...",maltohexaose transport,biological_process,maltohexaose,C01936,NaN,NaN,CHEBI:6667
1510,GO:2001142,"""The directed movement of a nicotinateacetate ...",nicotinate transport,biological_process,nicotinate,C00253,NaN,NaN,CHEBI:7559


In [19]:
df_GO_metabolite.to_pickle(join("..", "..", "data", "GOA", "go_terms", "GO_terms_with_sub_IDs.pkl"))